In [2]:
import numpy as np
import pandas as pd
!pip install ortools;
import ortools
from ortools.linear_solver import pywraplp
import pickle 

solver = pywraplp.Solver('LinearProgrammingExample',
                         pywraplp.Solver.GLOP_LINEAR_PROGRAMMING)


df = pd.read_pickle("dummy.pkl")

In [3]:
vectors = df["Vector"]

In [4]:
distances = np.zeros((2500,2500))
from scipy.spatial import distance

for i in range(len(vectors)):
    a = vectors[i]
    for j in range(len(vectors)):
        b = vectors[j]
        dst = distance.euclidean(a, b)
        distances[i,j] = dst

In [ ]:
#pickle 
#pickle.dump(distances, open("dist.pkl", "w"))

In [38]:
C = []
authors = df["Author"].unique()

for i in authors:
  df_sub = df[df["Author"] == i]
  C.append(df_sub.index.values)

In [36]:
def LP(λ):    
    x = []
    y = []
    a = 0.5
    a_1 = np.ceil(1/a)
    colors = 50
    k = 25
    lam = λ
    n = 2500
    solver = pywraplp.Solver('LP',
                             pywraplp.Solver.GLOP_LINEAR_PROGRAMMING)

    objective = solver.Objective()
    for i in range(n):
      x.append([])
      for j in range(n):
        x[i].append(solver.NumVar(0.0, 1.0,'x'))
        y.append(solver.NumVar(0.0, 1.0,'y'))
        objective.SetCoefficient(x[i][j], 1)
    objective.SetMaximization()

    constraint = []
    for j in range(n):
      constraint.append(solver.Constraint(1, 1))
      for i in range(n):
        constraint[j].SetCoefficient(x[i][j],1)

    for i in range(n):
      for j in range(n):
        constraint.append(solver.Constraint(-solver.infinity(), 0))
        constraint[-1].SetCoefficient(x[i][j],1)
        constraint[-1].SetCoefficient(y[i],-1)

    for i in range(colors):
      constraint.append(solver.Constraint(-solver.infinity(), 0))
      for j in range(n):
        constraint[-1].SetCoefficient(x[i][j],-a)
        if j in C[i]:
          constraint[-1].SetCoefficient(x[i][j],1-a)

    for i in range(n):
      constraint.append(solver.Constraint(0, solver.infinity()))
      for j in range(n):
        constraint[-1].SetCoefficient(x[i][j],1)
        constraint[-1].SetCoefficient(y[i],-a_1)

    constraint.append(solver.Constraint(0, k))
    for i in range(n):
      constraint[-1].SetCoefficient(y[i],1)

    for i in range(n):
      for j in range(n):
        if distances[i,j] > lam:
          constraint.append(solver.Constraint(0, 0))
          constraint[-1].SetCoefficient(x[i][j],1)

    status = solver.Solve()
    status == solver.OPTIMAL
    
    if solver.OPTIMAL == True:
        X_sol = np.zeros((n,n))
        Y_sol = []
        for i in range(n):
            Y_sol.append(y[i].solution_value())
            for j in range(n):
                X_sol[i,j] = x[i][j].solution_value()
        return (X_sol, Y_sol)
    else: 
        return False

In [ ]:
λ = 19/2
factor = 1.1

while LP(λ) == False: 
    λ = λ * factor

sol = LP(λ)

In [39]:
LP(13)

KeyboardInterrupt: 

In [14]:
x = pd.read_pickle("X15.pkl")
y = pd.read_pickle("Y15.pkl")
sol = (x,y)
n = 2500

In [15]:
v = 0
z = 0
for i in range(n):
  z += sol[1][i]
  for j in range(n):
    v+= sol[0][i,j]
print(v,z)

2500.0000000002515 24.999999999999865


In [16]:

v = 0
z = 0
F = []
for i in range(n):
    if sol[1][i] > 0:
        F.append(i)

dist = np.zeros((n,n))

for i in F:
    for j in F:
        if i != j:
            dist[i,j]=distances[i,j]


In [136]:
dist

array([[1.00000000e+06, 1.00000000e+06, 1.00000000e+06, ...,
        1.00000000e+06, 1.00000000e+06, 1.00000000e+06],
       [1.00000000e+06, 1.00000000e+06, 1.00000000e+06, ...,
        1.00000000e+06, 1.00000000e+06, 1.00000000e+06],
       [1.00000000e+06, 1.00000000e+06, 1.00000000e+06, ...,
        1.00000000e+06, 1.00000000e+06, 1.00000000e+06],
       ...,
       [1.00000000e+06, 1.00000000e+06, 1.00000000e+06, ...,
        1.00000000e+06, 1.11989784e+01, 1.92325687e+01],
       [1.00000000e+06, 1.00000000e+06, 1.00000000e+06, ...,
        1.11989784e+01, 1.00000000e+06, 2.12009296e+01],
       [1.00000000e+06, 1.00000000e+06, 1.00000000e+06, ...,
        1.92325687e+01, 2.12009296e+01, 1.00000000e+06]])

In [28]:
def greedy_lambda(F, F_distances, lamb):
    removed = set()
    Fcopy = F.copy()
    Fcopy1 = F.copy()
    for i in Fcopy:
        if i not in removed:
            for j in Fcopy:
                if j not in removed:
                    if (i == j):
                        pass
                    elif (F_distances[i,j] < 2*lamb):
                        Fcopy1.remove(j)
                        removed.add(j)
    return Fcopy1

In [25]:
def greedy_lam(F, dist, lam):
    removed = []
    Fcopy = F.copy()
    for i in F: 
        for j in F: 
            if i not in removed and j not in removed and i!=j: 
                if distances[i,j] < 2*lam: 
                    Fcopy.remove(j)
                    removed.append(j)
    return Fcopy

In [25]:
d = {}
for f in F:
    if f in F_prime: 
        d[f] = []
        d[f].append(f)
    else:
        for i in F_prime:
            if dist[f,i] < 2*lam:
                if i in d:
                    d[i].append(f)
                else:
                    d[i] = []
                    d[i].append(f)
                break


In [27]:
x_new = np.zeros((n,n))
for j in np.arange(n): 
    for i in F_prime: 
        for element in d[i]: 
            x_new[i,j] += sol[0][element,j]

{92: [92],
 93: [93],
 2415: [2415, 2493, 2497, 2498, 2499],
 179: [179],
 431: [431],
 513: [513],
 544: [544],
 677: [677],
 718: [718],
 956: [956],
 976: [976],
 1040: [1040],
 1293: [1293],
 1303: [1303],
 1634: [1634],
 2495: [2495],
 2496: [2496]}